In [2]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from matplotlib import pyplot

In [3]:
dataset17 = pd.read_csv("cbb-data/cbb17.csv")
dataset19 = pd.read_csv("cbb-data/cbb19.csv")
dataset20 = pd.read_csv("cbb-data/cbb20.csv")
print(dataset19.keys())

FileNotFoundError: [Errno 2] No such file or directory: 'cbb-data/cbb17.csv'

In [ ]:
cleaned17 = dataset17.dropna()
cleaned17 = cleaned17[cleaned17.POSTSEASON != "R68"]
#dummies = pd.get_dummies(cleaned17.CONF)
cleaned17 = cleaned17.replace({"R32": 1, "R64": 0, "S16": 2, "E8": 3, "F4": 4, "2ND": 5, "Champions": 6})
cleaned17['G'] = cleaned17['G'] - cleaned17['POSTSEASON'] - 1
index = (cleaned17[cleaned17['POSTSEASON']== 6].index.values)
cleaned17.at[1, 'G'] = cleaned17.at[index[0],'G'] + 1
cleaned17['W'] = cleaned17['W'] - cleaned17['POSTSEASON']

cleaned19 = dataset19.dropna()
cleaned19.at[49, 'CONF'] = 'ASun'
cleaned19 = cleaned19[cleaned19.POSTSEASON != "R68"]
#dummies2 = pd.get_dummies(cleaned19.CONF)
cleaned19 = cleaned19.replace({"R32": 1, "R64": 0, "S16": 2, "E8": 3, "F4": 4, "2ND": 5, "Champions": 6})
cleaned19['G'] = cleaned19['G'] - cleaned19['POSTSEASON'] - 1
index = (cleaned19[cleaned19['POSTSEASON']== 6].index.values)
cleaned19.at[index[0], 'G'] = cleaned19.at[index[0],'G'] + 1
cleaned19['W'] = cleaned19['W'] - cleaned19['POSTSEASON']

cleaned20 = dataset20.dropna()
cleaned20.drop(cleaned20.loc[65:352].index, inplace=True)
cleaned20.at[49, 'CONF'] = 'ASun'
cleaned20 = cleaned20.replace({"R32": 1, "R64": 0, "S16": 2, "E8": 3, "F4": 4, "2ND": 5, "Champions": 6})

In [ ]:
cleaned17 = cleaned17.drop(["TEAM"], axis=1)
cleaned17 = cleaned17.drop(["CONF"], axis=1)
cleaned17=(cleaned17-cleaned17.mean())/cleaned17.std()
#cleaned17 = cleaned17.join(dummies)
#cleaned17 = cleaned17.drop(["ACC"], axis=1)
outputs17 = cleaned17["POSTSEASON"]
cleaned17 = cleaned17.drop(["POSTSEASON"], axis=1)
cleaned17 = cleaned17.drop(["SEED"], axis=1)

teams19 = cleaned19["TEAM"]
cleaned19 = cleaned19.drop(["TEAM"], axis=1)
cleaned19 = cleaned19.drop(["CONF"], axis=1)
cleaned19=(cleaned19-cleaned19.mean())/cleaned19.std()
#cleaned19 = cleaned19.join(dummies2)
#cleaned19 = cleaned19.drop(["ACC"], axis=1)
outputs19 = cleaned19["POSTSEASON"]
cleaned19 = cleaned19.drop(["POSTSEASON"], axis=1)
cleaned19 = cleaned19.drop(["SEED"], axis=1)

teams20 = cleaned20["TEAM"]
cleaned20 = cleaned20.drop(["TEAM", "CONF", "RK"], axis=1)
cleaned20=(cleaned20-cleaned20.mean())/cleaned20.std()
cleaned20

In [ ]:
regr = linear_model.LinearRegression()
# Train the model using the training sets
regr.fit(cleaned17, outputs17)
# Make predictions using the testing set
ncaa_pred = regr.predict(cleaned19)

In [ ]:
# get importance
importance = regr.coef_
# summarize feature importance
for i,v in enumerate(importance):
	print('Feature: %0d, Score: %.5f' % (i,v))
# plot feature importance
pyplot.bar([x for x in range(len(importance))], importance)
pyplot.show()

In [ ]:
#features importances for 19
print('Features - Coefficients:')
print('-------------------------------')
for i in range(len(regr.coef_)):
    print(cleaned19.columns[i], ": ", regr.coef_[i])


# The mean squared error
print('\nMean squared error: %.2f'
      % mean_squared_error(outputs19, ncaa_pred))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f'
      % r2_score(outputs19, ncaa_pred))

In [ ]:
fullData = pd.read_csv("cbb-data/cbb.csv")
cleaned = fullData.dropna()
cleaned = cleaned[cleaned.POSTSEASON != "R68"]
cleaned = cleaned.replace({"R32": 1, "R64": 0, "S16": 2, "E8": 3, "F4": 4, "2ND": 5, "Champions": 6})
cleaned['G'] = cleaned['G'] - cleaned['POSTSEASON'] - 1
index = (cleaned[cleaned['POSTSEASON']== 6].index.values)
cleaned.at[index[0], 'G'] = cleaned.at[index[0],'G'] + 1
cleaned.at[index[1], 'G'] = cleaned.at[index[1],'G'] + 1
cleaned.at[index[2], 'G'] = cleaned.at[index[2],'G'] + 1
cleaned.at[index[3], 'G'] = cleaned.at[index[3],'G'] + 1
cleaned.at[index[4], 'G'] = cleaned.at[index[4],'G'] + 1
cleaned['W'] = cleaned['W'] - cleaned['POSTSEASON']
#cleaned20 = cleaned
cleaned20 = cleaned[cleaned.YEAR != 2020]
cleaned = cleaned[cleaned.YEAR != 2019]

cleaned = cleaned.drop(["TEAM"], axis=1)
cleaned = cleaned.drop(["YEAR"], axis=1)
cleaned = cleaned.drop(["CONF"], axis=1)
cleaned = cleaned.drop(["SEED"], axis=1)
cleaned=(cleaned-cleaned.mean())/cleaned.std()
outputs = cleaned["POSTSEASON"]
cleaned = cleaned.drop(["POSTSEASON"], axis=1)

In [ ]:
regr = linear_model.LinearRegression()
# Train the model using the training sets
regr.fit(cleaned, outputs)
# Make predictions using the testing set
ncaa_pred = regr.predict(cleaned19)

In [ ]:
#features importances for 19
print('Features - Coefficients:')
print('-------------------------------')
for i in range(len(regr.coef_)):
    print(cleaned19.columns[i], ": ", regr.coef_[i])

# The mean squared error
print('\nMean squared error: %.2f'
      % mean_squared_error(outputs19, ncaa_pred))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f'
      % r2_score(outputs19, ncaa_pred))

In [ ]:
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge

# find best params and score for Ridge
ridge = Ridge()

parameters = {'alpha': [1e-3, 1e-2, 1, 5, 10, 20]}

ridge_regressor = GridSearchCV(ridge, parameters, scoring="neg_mean_squared_error", cv=5)

ridge_regressor.fit(cleaned, outputs)

print(ridge_regressor.best_params_)
print(ridge_regressor.best_score_)

In [ ]:
ridge = Ridge(alpha=1e-3)
ridge.fit(cleaned,outputs)
predictions = ridge.predict(cleaned19)
#features importances for 19
print('Features - Coefficients:')
print('-------------------------------')
for i in range(len(ridge.coef_)):
    print(cleaned19.columns[i], ": ", ridge.coef_[i])
# The mean squared error
print('\nMean squared error: %.2f'
      % mean_squared_error(outputs19, predictions))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f'
      % r2_score(outputs19, predictions))

In [ ]:
from sklearn.linear_model import Lasso

# find best params and score for Lasso
lasso = Lasso()

parameters = {'alpha': [1e-3, 1e-2, 1, 5, 10, 20]}

lasso_regressor = GridSearchCV(lasso, parameters, scoring='neg_mean_squared_error', cv=10)

lasso_regressor.fit(cleaned, outputs)

print(lasso_regressor.best_params_)
print(lasso_regressor.best_score_)

In [ ]:
lasso = Lasso(alpha=1e-3)
lasso.fit(cleaned,outputs)
predictions = lasso.predict(cleaned19)
print('Coefficients: \n', lasso.coef_)
# The mean squared error
print('Mean squared error: %.2f'
      % mean_squared_error(outputs19, predictions))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f'
      % r2_score(outputs19, predictions))

In [ ]:
#random classifier, and fit
randTree = RandomForestRegressor(min_samples_split=20, random_state=5);
randTree.fit(cleaned,outputs)

In [ ]:
#get team name info to print
teams=teams19

labels=outputs19

cleaned19

In [ ]:
#probabilies for each Class

teams=teams.to_numpy()

#get predicted outcomes for each team and print!
outcomes=randTree.predict(cleaned19)

for i in range(len(teams)):
    print(teams[i], outcomes[i])

wins=plt.bar([x for x in range(len(outcomes))], outcomes)

In [ ]:
#featureImportances for 19
importances19 = randTree.feature_importances_
print('Features - Coefficients:')
print('-------------------------------')
for i in range(len(importances19)):
    print(cleaned19.columns[i], " - ", importances19[i])

#from sci learn metrics
print('Mean squared error: %.2f'
      %mean_squared_error(labels, outcomes)) 
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f'
      % r2_score(labels, outcomes))

In [ ]:
#predicting the 2020 winner using Random Forest Regression
outcomes=randTree.predict(cleaned20)

for i in range(len(teams20)-1):
    print(teams20[i], outcomes[i])

wins=plt.bar([x for x in range(len(outcomes))], outcomes)

In [ ]:
#featureImportances for 20
importances20 = randTree.feature_importances_
print('Features - Coefficients:')
print('-------------------------------')
for i in range(len(importances20)):
    print(cleaned20.columns[i], ": ", importances20[i])